In [3]:
import pandas as pd
df = pd.read_csv("spotify_millsongdata.csv")

In [4]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [5]:
df.tail(5)

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [7]:
df.shape

(57650, 4)

In [8]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [9]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [10]:
df.head(10)


,artist,song,text
0,Bob Marley,All In One,Bend down low \r\nLet me tell you what I know...
1,Dusty Springfield,How Can I Be Sure,How can I be sure? \r\nIn a world that's cons...
2,Paul Simon,Train In The Distance,She was beautiful as southern skies the night ...
3,Reba Mcentire,He Wants To Get Married,"(Sandy Knox, Anthony Little) \r\n \r\nHe bel..."
4,Ella Fitzgerald,Dream,Get in touch with that sundown fellow \r\nAs ...
5,Violent Femmes,Cold Canyon,Travel 'round the country \r\nLook out for wh...
6,Sia,Loved Me Back To Life,I was walking dead inside my head \r\nI could...
7,Loretta Lynn,Be Proud Of Your Man,Woman he love you can't you see so help him an...
8,Tool,Intolerance,I don't want to be hostile. \r\nI don't want ...
9,Frank Sinatra,Come Rain Or Come Shine,"I'm gonna love you, like nobody's loved you \..."


In [11]:
df['text'][0]

"Bend down low  \r\nLet me tell you what I know now  \r\nBend down low  \r\nLet me tell you what I know, what I know, what I know  \r\n  \r\nLong time, we no have no nice time  \r\nDoo-yoo-doo-dun-doo, yeah, think about that  \r\nLong, long, long, long time, we no have no nice time  \r\nDoo-yoo-doo-dun-doo, yeah, think about that, mm  \r\n  \r\nOne love, one heart  \r\nLet's get together and feel alright  \r\nOne love, hear my plea, one heart  \r\nGive thanks and praise to the Lord, and I will feel alright  \r\n  \r\nSimmer down, you lickin' too hot, so  \r\nSimmer down, soon you'll get dropped, so  \r\nSimmer down, man, you hear what I say?  \r\n  \r\nAfter he  \r\nBreaks your heart  \r\nThen you'll be sad, so sad  \r\nAnd then your teardrops start  \r\nI tell you, then you'll know how  \r\nIt hurts to be alone  \r\n  \r\nOh, what a (... feeling ...), what a feeling (... be blue)  \r\nOh, what a feeling, oh, what a feeling  \r\n(Oh, what a feeling to be blue)  \r\nHave you ever had a 

In [12]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [21]:
import pandas as pd
import nltk
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pickle

# Load dataset
df = pd.read_csv("spotify_millsongdata.csv")
df = df.sample(5000).drop('link', axis=1).reset_index(drop=True)

# Text Cleaning/Preprocessing
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex=True)

stemmer = PorterStemmer()
def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

df['text'] = df['text'].apply(lambda x: tokenization(x))

# TF-IDF Vectorization
tfidvector = TfidfVectorizer(analyzer='word', stop_words='english')
matrix = tfidvector.fit_transform(df['text'])

# Cosine Similarity Calculation
similarity = cosine_similarity(matrix)

# Recommendation Function
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])), reverse=True, key=lambda x: x[1])
    
    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

# Serialize Model
pickle.dump(similarity, open('similarity.pkl', 'wb'))
pickle.dump(df, open('df.pkl', 'wb'))


In [20]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True